In [6]:
pip install cv2

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [5]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split

# Paths
data_dir = r"C:\Users\nages\AutonomousEV\SteeringAngle\IMG"
log_path = r"C:\Users\nages\AutonomousEV\SteeringAngle\driving_log.csv"

# Load CSV
df = pd.read_csv(log_path, header=None)
print("Total samples:", len(df))

# Function to fix image paths (in CSV they may have full paths from simulator)
def fix_path(img_path):
    filename = os.path.basename(img_path.strip())
    return os.path.join(data_dir, filename)

# Apply path correction to center, left, right
df[0] = df[0].apply(fix_path)  # center
df[1] = df[1].apply(fix_path)  # left
df[2] = df[2].apply(fix_path)  # right

# Extract steering angles
df[3] = df[3].astype(float)


ModuleNotFoundError: No module named 'cv2'

In [ ]:
import cv2
import numpy as np
import sklearn
from sklearn.utils import shuffle

def preprocess(img):
    # crop (remove sky & hood)
    h, w, _ = img.shape
    img = img[int(h*0.35):int(h*0.9), :, :]
    # resize
    img = cv2.resize(img, (200,66))
    # normalize
    img = img / 255.0
    return img

def generator(samples, batch_size=32, img_dir="IMG"):
    num_samples = len(samples)
    while True:
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images, angles = [], []
            for batch_sample in batch_samples:
                center, left, right, steering = batch_sample[0], batch_sample[1], batch_sample[2], float(batch_sample[3])

                # correction factor for left/right
                correction = 0.2
                path_center = os.path.join(img_dir, center)
                path_left   = os.path.join(img_dir, left)
                path_right  = os.path.join(img_dir, right)

                img_center = preprocess(cv2.imread(path_center))
                img_left   = preprocess(cv2.imread(path_left))
                img_right  = preprocess(cv2.imread(path_right))

                images.extend([img_center, img_left, img_right])
                angles.extend([steering, steering+correction, steering-correction])

                # flipped images
                images.extend([np.fliplr(img_center)])
                angles.extend([-steering])

            X, y = np.array(images), np.array(angles)
            yield sklearn.utils.shuffle(X, y)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout

def nvidia_model():
    model = Sequential([
        Conv2D(24,(5,5),strides=(2,2),activation="relu", input_shape=(66,200,3)),
        Conv2D(36,(5,5),strides=(2,2),activation="relu"),
        Conv2D(48,(5,5),strides=(2,2),activation="relu"),
        Conv2D(64,(3,3),activation="relu"),
        Conv2D(64,(3,3),activation="relu"),
        Flatten(),
        Dense(100, activation="relu"),
        Dropout(0.5),
        Dense(50, activation="relu"),
        Dense(10, activation="relu"),
        Dense(1)  # output steering
    ])
    model.compile(loss="mse", optimizer="adam")
    return model


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
pip install tensorflow -i https://pypi.org/simple


In [ ]:
pip install tensorflow-cpu


^C
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 2.3.2 which is incompatible.
scipy 1.11.1 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.3.2 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-6.32.0-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached grpcio-1.74.0-cp311-cp311-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.11.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached numpy-2.3.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached h